# Global alignement : Needleman-wunsch

In [7]:
import numpy as np
from Bio.SubsMat import MatrixInfo

def needleman_wunsch(seq1, seq2, gap, mat):
    n = len(seq1) + 1  # Dimension of matrix column
    m = len(seq2) + 1  # Dimension of matrix row
    align_matrix = np.zeros((m, n), dtype=int)  # Create a matrix of dimensions (n,m)
    #matrix = MatrixInfo.blosum62 # Import the matrix blosum62.
    matrix = getattr(MatrixInfo,mat) # Retrieve the matrix referenced.
    # Fill the first row elements of the matrix with multiple of gap penalty.
    for i in range(m):
        align_matrix[i][0] = gap * i
        
    # Fill the first column elements of the matrix with multiple of gap penalty.    
    for j in range(n):
        align_matrix[0][j] = gap * j
        
    # Fill the rest of the matrix.
    for i in range(1, m):
        for j in range(1, n):
            try:
                m = matrix[seq1[j-1], seq2[i-1]] # As our keys are only in one way, if we receive a KeyError, we'll switch our key order.
            except KeyError:
                m = matrix[seq2[i-1], seq1[j-1]]
            di = align_matrix[i-1][j-1] + m # Compute the diagonal line
            ho = align_matrix[i][j-1] + gap # Compute the horizontal line
            ve = align_matrix[i-1][j] + gap # Compute the vertical line
            align_matrix[i][j] = max(di, ho, ve) # Choose the max value for the previously computed values.

    i = len(seq2) 
    j = len(seq1) 
    seq2_t = '>'
    seq1_s = '>'
    
    # Traceback our matrix starting from the end.
    while i > 0 and j > 0:
        try:
            m = matrix[seq1[j-1], seq2[i-1]]
        except KeyError:
            m = matrix[seq2[i-1], seq1[j-1]]
            
        if align_matrix[i, j] - m == align_matrix[i-1, j-1]: # Compute the diagonal line
            seq2_t = seq2_t + seq2[i-1]
            seq1_s = seq1_s + seq1[j-1]
            i = i-1
            j = j-1

        elif align_matrix[i, j] - gap == align_matrix[i, j-1]: # Compute the vertical line
            seq1_s = seq1_s + seq1[j-1]
            seq2_t = seq2_t + '_'
            j = j-1
            
        elif align_matrix[i, j] - gap == align_matrix[i-1, j]: # Compute the horizontal line
            seq1_s = seq1_s + '_'
            seq2_t = seq2_t + seq2[i-1]
            i = i-1
    # If i or j still are > 0, then we'll add enough gaps to arrive at the start of our sequence.        
    if i > 0:
        while i > 0:
            seq1_s = seq1_s + '_'
            seq2_t = seq2_t + seq2[i-1]
            i = i -1
            
    elif j > 0:
        while j > 0:
            seq1_s = seq1_s + seq1[j-1]
            seq2_t = seq2_t + '_'
            j = j -1

    print(seq2_t[::-1])
    print(seq1_s[::-1])
    
needleman_wunsch(seq1="MGGETFA", seq2 ="GGVTTF", gap=-4, mat = "blosum62")


_GGVTTF_>
MGG_ETFA>
